In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [ ]:
data_dir = '../input/intel-image-classification'

random_transforms = [transforms.RandomRotation(25),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.ColorJitter(0.5, 0.5, 0.5, 0.5)]
train_transforms = transforms.Compose([transforms.Resize(140),
                                 transforms.CenterCrop(128),
                                 transforms.RandomApply(random_transforms, p=0.6),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(140),
                                 transforms.CenterCrop(128),
                                 transforms.ToTensor()])

train_data = datasets.ImageFolder(data_dir + '/seg_train/seg_train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/seg_test/seg_test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)
output_classes = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [ ]:
data_iter = iter(trainloader)

images, labels = data_iter.next()
images = images.numpy()
fig = plt.figure(figsize=(10,10))
for ii in range(4):
    ax = fig.add_subplot(1, 5, ii+1, xticks=[], yticks=[])
    ax.set_title(output_classes[labels[ii]])
    plt.imshow(np.transpose(images[ii], (1, 2, 0)))

In [ ]:
model = models.densenet121(pretrained=True)

In [ ]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.2)),
                          ('fc2', nn.Linear(500, 6)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

is_cuda = torch.cuda.is_available()
print(is_cuda)

In [ ]:
criterion = nn.NLLLoss()
# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

if is_cuda:
    model.cuda()
epochs = 10
for e in range(epochs):
    running_loss = 0
    for inputs, labels in trainloader:
        # Move input and label tensors to the device
        if is_cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
        
        optimizer.zero_grad()
        
        # training
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else :
        with torch.no_grad():
            model.eval()
            test_loss = 0
            for images, labels in testloader:
                if is_cuda:
                    images, labels = images.cuda(), labels.cuda()
                test_output = model(images)
                test_loss += criterion(test_output, labels)
                top_p, top_class = test_output.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy = torch.mean(equals.type(torch.FloatTensor))
                print(f'Accuracy: {accuracy.item()*100}%')
        model.train()